# Estimate Binomial draw probabilility

In [1]:
using StatisticalRethinking
using DynamicHMC, TransformVariables, LogDensityProblems, MCMCDiagnostics
using Parameters, ForwardDiff

Define a structure to hold the data.

In [2]:
struct BernoulliProblem
    "Total number of draws in the data."
    n::Int
    "Number of draws `==1` in the data"
    s::Vector{Int}
end;

Make the type callable with the parameters *as a single argument*.

In [3]:
function (problem::BernoulliProblem)((α, )::NamedTuple{(:α, )})
    @unpack n, s = problem        # extract the data
    loglikelihood(Binomial(n, α), s)
end

Create the data and complete setting up the problem.

In [4]:
obs = rand(Binomial(9, 2/3), 1)
p = BernoulliProblem(9, obs)
p((α = 0.5, ))

-2.6548056865833978

Write a function to return properly dimensioned transformation.

In [5]:
problem_transformation(p::BernoulliProblem) =
    as((α = as𝕀, ),  )
# Use a flat priors (the default, omitted) for α
P = TransformedLogDensity(problem_transformation(p), p)
∇P = LogDensityRejectErrors(ADgradient(:ForwardDiff, P));

Sample

In [6]:
chain, NUTS_tuned = NUTS_init_tune_mcmc(∇P, 1000)

MCMC, adapting ϵ (75 steps)
0.00098 s/step ...done
MCMC, adapting ϵ (25 steps)
6.9e-6 s/step ...done
MCMC, adapting ϵ (50 steps)
0.0017 s/step ...done
MCMC, adapting ϵ (100 steps)
8.9e-6 s/step ...done
MCMC, adapting ϵ (200 steps)
8.2e-6 s/step ...done
MCMC, adapting ϵ (400 steps)
7.2e-6 s/step ...done
MCMC, adapting ϵ (50 steps)
9.9e-6 s/step ...done
MCMC (1000 steps)
1.0e-5 s/step ...done


(NUTS_Transition{Array{Float64,1},Float64}[NUTS_Transition{Array{Float64,1},Float64}([1.73514], -3.49102, 2, DoubledTurn, 0.926352, 3), NUTS_Transition{Array{Float64,1},Float64}([1.46616], -3.35604, 1, DoubledTurn, 1.0, 1), NUTS_Transition{Array{Float64,1},Float64}([2.00589], -3.85481, 1, AdjacentTurn, 0.905746, 3), NUTS_Transition{Array{Float64,1},Float64}([0.0613507], -3.89369, 2, AdjacentTurn, 0.988446, 7), NUTS_Transition{Array{Float64,1},Float64}([0.988413], -4.10344, 2, DoubledTurn, 0.971213, 3), NUTS_Transition{Array{Float64,1},Float64}([0.499499], -3.19531, 1, AdjacentTurn, 0.937363, 3), NUTS_Transition{Array{Float64,1},Float64}([0.118686], -5.335, 2, DoubledTurn, 0.654695, 3), NUTS_Transition{Array{Float64,1},Float64}([-0.147832], -4.59734, 1, DoubledTurn, 0.815612, 1), NUTS_Transition{Array{Float64,1},Float64}([0.869083], -4.1853, 1, AdjacentTurn, 1.0, 3), NUTS_Transition{Array{Float64,1},Float64}([0.869083], -4.16711, 1, AdjacentTurn, 0.742771, 3)  …  NUTS_Transition{Array{F

To get the posterior for ``α`` use `get_position` and then transform back.

In [7]:
posterior = TransformVariables.transform.(Ref(problem_transformation(p)), get_position.(chain));

Extract the parameter.

In [8]:
posterior_α = first.(posterior);

check the effective sample size

In [9]:
ess_α = effective_sample_size(posterior_α)

379.84849701821923

NUTS-specific statistics

In [10]:
NUTS_statistics(chain)

Hamiltonian Monte Carlo sample of length 1000
  acceptance rate mean: 0.93, min/25%/median/75%/max: 0.18 0.91 0.98 1.0 1.0
  termination: AdjacentTurn => 40% DoubledTurn => 60%
  depth: 1 => 61% 2 => 39%


check the mean

In [11]:
mean(posterior_α)

0.7305950782767006

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*